# Analyse des Logs Android avec PySpark
## Par OUSSAMA ELMAKNASSI

Ce notebook analyse les logs Android en utilisant PySpark pour compter :
- Les messages par **niveau** (I, D, E, W, V)
- Les messages par **heure**
- Les messages par **minute**

## 1. Installation et Configuration de PySpark

In [ ]:
# Installation de PySpark
!pip install pyspark

# Installation de Java (déjà disponible sur Colab)
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark"

print("✓ PySpark installé et configuré")

## 2. Upload du fichier Android.log

**Instructions :**
1. Exécutez la cellule ci-dessous
2. Cliquez sur "Choose Files"
3. Sélectionnez votre fichier `Android.log`

In [ ]:
from google.colab import files
import io

print("📁 Veuillez uploader votre fichier Android.log...")
uploaded = files.upload()

# Sauvegarder le fichier
for filename in uploaded.keys():
    print(f"✓ Fichier uploadé: {filename} ({len(uploaded[filename])} bytes)")
    with open('Android.log', 'wb') as f:
        f.write(uploaded[filename])

print("\n✓ Fichier prêt pour l'analyse !")

## 3. Initialisation de Spark

In [ ]:
from pyspark import SparkContext, SparkConf
import re

# Configuration de Spark
conf = SparkConf().setAppName("Android Log Analysis").setMaster("local[*]")

# Arrêter le contexte existant s'il y en a un
try:
    sc.stop()
except:
    pass

# Créer un nouveau contexte Spark
sc = SparkContext(conf=conf)

print("✓ SparkContext initialisé")
print(f"  - Version Spark: {sc.version}")
print(f"  - Master: {sc.master}")
print(f"  - Application: {sc.appName}")

## 4. Chargement des données

In [ ]:
# Charger le fichier de logs dans un RDD
log_RDD = sc.textFile("Android.log")

# Compter le nombre total de lignes
total_lines = log_RDD.count()
print(f"✓ Fichier chargé: {total_lines:,} lignes")

# Afficher quelques exemples
print("\n📋 Exemples de lignes:")
for i, line in enumerate(log_RDD.take(5), 1):
    print(f"  {i}. {line[:100]}...")

## 5. Parsing des logs

In [ ]:
def parse_log_line(line):
    """Extrait le timestamp et le level d'une ligne de log"""
    try:
        # Chercher le timestamp
        timestamp_match = re.search(r'(\d{4}-\d{2}-\d{2}[T ]\d{2}:\d{2})|' +
                                   r'(\d{2}-\d{2} \d{2}:\d{2})|' +
                                   r'(\d{2}:\d{2}:\d{2})', line)
        
        # Chercher le level (E, W, I, D, V)
        level_match = re.search(r'\b([EWIDV])\b|\[(ERROR|WARN|WARNING|INFO|DEBUG|VERBOSE)\]', 
                               line, re.IGNORECASE)
        
        timestamp = timestamp_match.group(0) if timestamp_match else None
        level = level_match.group(1) or level_match.group(2) if level_match else 'UNKNOWN'
        
        return (timestamp, level.upper())
    except:
        return (None, 'UNKNOWN')

def extract_hour(timestamp):
    """Extrait l'heure d'un timestamp"""
    if not timestamp:
        return 'UNKNOWN'
    try:
        hour_match = re.search(r'(\d{2}):\d{2}', timestamp)
        return hour_match.group(1) + ':00' if hour_match else 'UNKNOWN'
    except:
        return 'UNKNOWN'

def extract_minute(timestamp):
    """Extrait l'heure et la minute d'un timestamp"""
    if not timestamp:
        return 'UNKNOWN'
    try:
        minute_match = re.search(r'(\d{2}:\d{2})', timestamp)
        return minute_match.group(1) if minute_match else 'UNKNOWN'
    except:
        return 'UNKNOWN'

# Parser toutes les lignes
parsed_RDD = log_RDD.map(parse_log_line)
print("✓ Fonctions de parsing définies")
print("✓ RDD parsé créé")

## 6. COUNT PAR NIVEAU (LEVEL)

In [ ]:
print("="*60)
print("COUNT PAR NIVEAU (LEVEL)")
print("="*60)

# Compter par niveau
level_counts = parsed_RDD.map(lambda x: (x[1], 1)) \
                        .reduceByKey(lambda a, b: a + b) \
                        .sortBy(lambda x: x[1], ascending=False)

level_results = level_counts.collect()

# Afficher les résultats
for level, count in level_results:
    print(f"{level}: {count:,} messages")

print(f"\nTotal: {sum([count for _, count in level_results]):,} messages")

# Sauvegarder
level_counts.map(lambda x: f"{x[0]}: {x[1]}").saveAsTextFile("count_by_level")
print("\n✓ Sauvegardé dans: count_by_level/")

## 7. COUNT PAR HEURE

In [ ]:
print("\n" + "="*60)
print("COUNT PAR HEURE")
print("="*60)

# Compter par heure
hour_counts = parsed_RDD.map(lambda x: (extract_hour(x[0]), 1)) \
                       .reduceByKey(lambda a, b: a + b) \
                       .sortBy(lambda x: x[0])

hour_results = hour_counts.collect()

# Afficher les résultats
for hour, count in hour_results:
    print(f"{hour}: {count:,} messages")

print(f"\nTotal: {sum([count for _, count in hour_results]):,} messages")

# Sauvegarder
hour_counts.map(lambda x: f"{x[0]}: {x[1]}").saveAsTextFile("count_by_hour")
print("\n✓ Sauvegardé dans: count_by_hour/")

## 8. COUNT PAR MINUTE

In [ ]:
print("\n" + "="*60)
print("COUNT PAR MINUTE (Top 50)")
print("="*60)

# Compter par minute
minute_counts = parsed_RDD.map(lambda x: (extract_minute(x[0]), 1)) \
                         .reduceByKey(lambda a, b: a + b) \
                         .sortBy(lambda x: x[1], ascending=False)

# Afficher le top 50
minute_results = minute_counts.take(50)
for minute, count in minute_results:
    print(f"{minute}: {count:,} messages")

total_minutes = minute_counts.count()
print(f"\nNombre total de minutes différentes: {total_minutes:,}")

# Sauvegarder
minute_counts.map(lambda x: f"{x[0]}: {x[1]}").saveAsTextFile("count_by_minute")
print("\n✓ Sauvegardé dans: count_by_minute/")

## 9. Statistiques globales

In [ ]:
print("\n" + "="*60)
print("STATISTIQUES GLOBALES")
print("="*60)

total_logs = log_RDD.count()
print(f"Nombre total de lignes de log: {total_logs:,}")

# Statistiques sur les minutes
minute_stats = minute_counts.map(lambda x: x[1]).stats()
print(f"\nMessages par minute:")
print(f"  - Moyenne: {minute_stats.mean():,.2f}")
print(f"  - Maximum: {minute_stats.max():,}")
print(f"  - Minimum: {minute_stats.min():,}")

# Statistiques sur les heures
hour_stats = hour_counts.map(lambda x: x[1]).stats()
print(f"\nMessages par heure:")
print(f"  - Moyenne: {hour_stats.mean():,.2f}")
print(f"  - Maximum: {hour_stats.max():,}")
print(f"  - Minimum: {hour_stats.min():,}")

# Répartition par niveau
print(f"\nRépartition par niveau:")
total = sum([count for _, count in level_results])
for level, count in level_results:
    percentage = (count / total) * 100
    print(f"  - {level}: {percentage:.2f}%")

print("\n" + "="*60)
print("ANALYSE TERMINÉE !")
print("="*60)

## 10. Télécharger les résultats

In [ ]:
# Créer un fichier ZIP avec tous les résultats
!zip -r results.zip count_by_level count_by_hour count_by_minute

# Télécharger le fichier ZIP
from google.colab import files
files.download('results.zip')

print("✓ Résultats téléchargés !")

## 11. Nettoyage

In [ ]:
# Arrêter le contexte Spark
sc.stop()
print("✓ SparkContext arrêté")